# Web Scraping from IMDb and Rotten Tomatoes

In [1]:
import pandas as pd

In [2]:
#get all titles from static netflix data
    netflix = pd.read_excel("/Users/abhiniti/OneDrive - The University of Chicago/Spring 2020/Data Engineering Platforms - MSCA 31012/Final Project/Netflix/tables_netflix.xlsx")
netflix_titles = [netflix["title"]]
netflix_df = pd.DataFrame(netflix_titles).transpose().drop_duplicates()
#netflix_df

In [3]:
#using imdbpy
import imdb
moviesDB = imdb.IMDb()

In [4]:
#beautiful soup for Rotten Tomatoes
from bs4 import BeautifulSoup
import requests
import re  # for url

In [5]:
#get total count of titles
count = pd.DataFrame(netflix_df).shape[0]
count

3756

In [6]:
df=[]
df = pd.DataFrame(columns = {"title", "imdb_user_review", "imdb_critic_review","tomato_critic_review","tomato_user_review"})
columns = ['title', 'imdb_user_review', 'imdb_critic_review',"tomato_critic_review","tomato_user_review"]
data=[]

for x in range(0, count):
    #get title from netflix dataset
    input_title = netflix_df.iloc[x]["title"]
    #print(input_title)

    ############search in IMDb    
    try:
        movies = moviesDB.search_movie(input_title)
        id = movies[0].getID()
        movie = moviesDB.get_movie(id,info=['main', 'plot', 'reviews', 'critic reviews'])
        #get title
        title = movie["title"]
        #get user review
        try:
            imdb_user_review = movie["rating"]
        except:
            imdb_user_review = -999
        #get critic review
        try:
            imdb_critic_review = movie["metascore"]
        except:
            imdb_critic_review = -999
            
    except:
        print(input_title + " was not found on IMDb")
        
    ############search in Rotten Tomatoes
    try: 
        try: #first try in movies
            tomato_url = "https://www.rottentomatoes.com/m/" + re.sub(':', '', re.sub(' ', '_', input_title))
            soup = BeautifulSoup(requests.get(tomato_url).text)
            try:
                tomato_critic_review = soup.find('span', {'class': 'meter-value'}).contents[0]
            except:
                tomato_critic_review = -999
            try:
                tomato_user_review = soup.find('span', {'class': 'mop-ratings-wrap__percentage'}).contents[0]
                tomato_user_review = tomato_user_review.replace("\n","").replace(" ","").replace("%","")
            except:
                tomato_user_review = -999
        except: #then try in tv shows
            tomato_url = "https://www.rottentomatoes.com/tv/" + re.sub(':', '', re.sub(' ', '_', input_title))
            soup = BeautifulSoup(requests.get(tomato_url).text)
            try:
                tomato_critic_review = soup.find('span', {'class': 'meter-value'}).contents[0]
            except:
                tomato_critic_review = -999
            try:
                tomato_user_review = soup.find('span', {'class': 'mop-ratings-wrap__percentage'}).contents[0]
                tomato_user_review = tomato_user_review.replace("\n","").replace(" ","").replace("%","")
            except:
                tomato_user_review = -999
    except:
        tomato_critic_review = -999
        tomato_user_review = -999
        print(input_title + " was not found on Rotten Tomatoes")
    
    #append to dataframe
    values = [title,imdb_user_review,imdb_critic_review,tomato_critic_review,tomato_user_review]
    zipped = zip(columns, values)
    a_dictionary = dict(zipped)
    data.append(a_dictionary)

    
#print(data)
    
    

In [7]:
final = pd.DataFrame(data)

In [12]:
final[["title","imdb_user_review","tomato_user_review"]]

,title,imdb_user_review,tomato_user_review
0,Norm of the North: King Sized Adventure,3.1,36
1,#REALITYHIGH,5.2,-999
2,Automata,6.1,30
3,Fabrizio Copano: Solo pienso en mi,4.9,-999
4,Good People,5.5,-999


In [93]:
final.to_csv('/Users/abhiniti/OneDrive - The University of Chicago/Spring 2020/Data Engineering Platforms - MSCA 31012/Final Project/Netflix/Step 2 Data Storage/reviews.csv')